In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Conv3D, Conv2DTranspose, Conv3DTranspose, Flatten, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from PIL import Image
import Hyperparameters as hyp
import os
import sys

In [2]:
train_path = 'Data/Training2/'
test_path = '../input/maxcs229test/'

In [3]:
lr_opts = [0.001, 0.0001]
decay_opts = [0.0001, 0.00001]
eps_opts = [0.00001, 0.000001]

In [4]:
#--------------------------- SlidingWindow.py ---------------------------#
def get_sliding_windows(frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    frames_list : list
        A list of sorted frames of shape Config.FRAME_RES X Config.FRAME_RES
    sequence_size: int
        The size of the lstm sequence
    Returns
    -------
    list
        A list of clips , 20 frames each
    """
    clips = []
    for i in range(len(frames_list) - sequence_size):
        clip = np.zeros(shape=(sequence_size, hyp.FRAME_RES, hyp.FRAME_RES, 1))
        clip[:, :, :, 0] = frames_list[i : i + sequence_size]
        clips.append(clip)
    return np.array(clips)

def getData(path):
    data = []
    for file in os.listdir(path):
        if file[-3:] == "npy":
            all_frames = np.load(path + file, allow_pickle=True)
            data.extend(get_sliding_windows(frames_list=all_frames, sequence_size=hyp.CLIP_LEN))
            
    return np.array(data)

In [5]:
def experiment1(train_flat, val_flat):
    model = Sequential()
    model.add(Dense(int(train_flat.shape[1] / 2), activation="relu", input_shape=(train_flat.shape[1],)))
    model.add(Dense(train_flat.shape[1]))
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=hyp.lr['Base_Auto'], decay=hyp.decay['Base_Auto'], epsilon=hyp.eps['Base_Auto']), loss="mse")
    model.summary()
    es = EarlyStopping(min_delta=hyp.eps['Base_Auto'], verbose=1, patience=3)
    history = model.fit(train_flat, train_flat, validation_data=(val_flat, val_flat), epochs=10000, callbacks=[es])
    model.save("Base_Auto")
    np.save("Base_Loss", np.array([history.history['val_loss'], history.history['loss']]))

In [ ]:
train_data = getData(train_path)
train_data, val_data, _,  _ = train_test_split(train_data, train_data, test_size=0.1)
train_flat = np.array([np.ravel(train_data[i]) for i in range(train_data.shape[0])])
val_flat = np.array([np.ravel(val_data[i]) for i in range(val_data.shape[0])])
print(f"Train_data shape is {train_data.shape}")
experiment1(train_flat, val_flat)

Train_data shape is (40583, 10, 32, 32, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5120)              52433920  
_________________________________________________________________
dense_1 (Dense)              (None, 10240)             52439040  
Total params: 104,872,960
Trainable params: 104,872,960
Non-trainable params: 0
_________________________________________________________________
Train on 40583 samples, validate on 4510 samples
Epoch 1/10000
40583/40583 [==============================] - 57s 1ms/sample - loss: 1500.9742 - val_loss: 1232.5187
Epoch 2/10000
40583/40583 [==============================] - 57s 1ms/sample - loss: 1151.3873 - val_loss: 1086.4935
Epoch 3/10000
40583/40583 [==============================] - 58s 1ms/sample - loss: 1040.6515 - val_loss: 1007.6048
Epoch 4/10000
40583/40583 [==============================] - 57s

In [ ]:
# Tune hyperparameters
val_loss = sys.maxsize
best_lr, best_decay, best_eps = 0, 0, 0
for lr in lr_opts:
    for decay in decay_opts:
        for eps in eps_opts:          
            model = Sequential()
            model.add(Dense(int(train_flat.shape[1] / 2), activation="relu", input_shape=(train_flat.shape[1],)))
            model.add(Dense(train_flat.shape[1]))
            model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr, decay=decay, epsilon=eps), loss="mse")
            model.summary()
            es = EarlyStopping(min_delta=eps, verbose=1, patience=3)
            history = model.fit(train_flat, train_flat, validation_data=(val_flat, val_flat), epochs=10000, callbacks=[es])
            val = history.history['val_loss']
            test = history.history['loss']
            if val[-1] < val_loss:
                val_loss = val[-1]
                best_lr = lr
                best_decay = decay
                best_eps = eps
                model.save("Base_Auto")
                np.save("Base_Loss", np.array([val, test]))
print('The optimal training hyperparams are:')
print(f"lr = {best_lr}")
print(f"decay = {best_decay}")
print(f"eps = {best_eps}")